<a target="_blank" href="https://colab.research.google.com/github/ignaciomsarmiento/Urban_Slides/blob/main/Lecture14/Notebook_Example_Hedonic_Prices.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


##  House Prices Indices

One of the most common approaches for constructing house price indices are hedonic price functions:

\begin{align}
log(P)_{it} = \alpha + \sum_{t=1}^T \delta_t D_{it} + \sum_{j=1}^h \beta_j H_{ij} + \sum_{k=1}^n \beta_k N_{ik}  + u_{it}
\end{align}


- $log(P)_{it}$ represents the natural logarithm of the sale price of house $i$ at time $t$ $(t = 1,\dots, T)$, 
- $D_{it}$ is a variable indicating the house $i$ sold at time $t$,
- $H$ represents structural and property characteristics of the house (e.g. square footage of the living area, lot size, etc) 
-  $N$ represents location characteristics, (e.g. quality of schools,distance to CBD, parks, etc ) 


### The Ames Housing Data

For this exersise we are going to use housing data from Ames, Iowa, available on the `modeldata` package.

Let's load the packages:

In [1]:
# install.packages("pacman") #run this line if you use Google Colab

In [2]:
#packages
require("pacman")
p_load("tidyverse", #data wrangling
       "modeldata", # package with the housing data from Ames, Iowa
       "stargazer", # gazing at the stars 
       "broom", #tidy data set
       "fixest" #fix effect estimation
        ) 


Loading required package: pacman



 And the data set:

In [3]:
data("ames", package = "modeldata")

The Ames housing data is a normal [tibble](https://tibble.tidyverse.org/).

In [4]:
head(ames)

MS_SubClass,MS_Zoning,Lot_Frontage,Lot_Area,Street,Alley,Lot_Shape,Land_Contour,Utilities,Lot_Config,⋯,Fence,Misc_Feature,Misc_Val,Mo_Sold,Year_Sold,Sale_Type,Sale_Condition,Sale_Price,Longitude,Latitude
<fct>,<fct>,<dbl>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<int>,<int>,<int>,<fct>,<fct>,<int>,<dbl>,<dbl>
One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,141,31770,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,⋯,No_Fence,None,0,5,2010,WD,Normal,215000,-93.61975,42.05403
One_Story_1946_and_Newer_All_Styles,Residential_High_Density,80,11622,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,⋯,Minimum_Privacy,None,0,6,2010,WD,Normal,105000,-93.61976,42.05301
One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,81,14267,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,⋯,No_Fence,Gar2,12500,6,2010,WD,Normal,172000,-93.61939,42.05266
One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,93,11160,Pave,No_Alley_Access,Regular,Lvl,AllPub,Corner,⋯,No_Fence,None,0,4,2010,WD,Normal,244000,-93.61732,42.05125
Two_Story_1946_and_Newer,Residential_Low_Density,74,13830,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Inside,⋯,Minimum_Privacy,None,0,3,2010,WD,Normal,189900,-93.63893,42.06090
Two_Story_1946_and_Newer,Residential_Low_Density,78,9978,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Inside,⋯,No_Fence,None,0,6,2010,WD,Normal,195500,-93.63893,42.06078


The description of the variables can be viewed here: https://jse.amstat.org/v19n3/decock/DataDocumentation.txt

### Modelling Time Dummies

In [5]:
table(ames$Year_Sold)



2006 2007 2008 2009 2010 
 625  694  622  648  341 

In [6]:
class(ames$Year_Sold)

[1] "integer"

In [7]:
ames<- ames  %>% mutate(year=factor(Year_Sold,levels=c(2006,2007,2008,2009,2010),
                                    labels=c("d2006","d2007","d2008","d2009","d2010")))

In [8]:
table(ames$year)


d2006 d2007 d2008 d2009 d2010 
  625   694   622   648   341 

In [9]:
class(ames$year)

[1] "factor"

In [10]:
reg1<-lm(Sale_Price ~year+ Gr_Liv_Area  + Bldg_Type ,data=ames)
stargazer(reg1,type="text")


                        Dependent variable:    
                    ---------------------------
                            Sale_Price         
-----------------------------------------------
yeard2007                    1,268.544         
                            (2,984.665)        
                                               
yeard2008                   -1,933.982         
                            (3,063.130)        
                                               
yeard2009                     -70.717          
                            (3,034.576)        
                                               
yeard2010                   -3,071.273         
                            (3,641.608)        
                                               
Gr_Liv_Area                 113.814***         
                              (1.998)          
                                               
Bldg_TypeTwoFmCon         -59,017.220***       
                            (6,961.688)

In [11]:
ames<- ames  %>% mutate(log_Sale_Price=log(Sale_Price))

In [12]:
ames<- ames  %>% mutate(d2006=ifelse(Year_Sold==2006,1,0),
                        d2007=ifelse(Year_Sold==2007,1,0),
                        d2008=ifelse(Year_Sold==2008,1,0),
                        d2009=ifelse(Year_Sold==2009,1,0),
                        d2010=ifelse(Year_Sold==2010,1,0)
                       )

## Regressions 

In [13]:
reg2<-lm(log_Sale_Price ~year+ Gr_Liv_Area  + Bldg_Type ,data=ames)

reg3<-lm(log_Sale_Price ~d2007+d2008+d2009+d2010+ Gr_Liv_Area  + Bldg_Type ,data=ames)

stargazer(reg2,reg3,type="text")


                                    Dependent variable:     
                                ----------------------------
                                       log_Sale_Price       
                                     (1)            (2)     
------------------------------------------------------------
yeard2007                           0.015                   
                                   (0.015)                  
                                                            
yeard2008                           -0.006                  
                                   (0.016)                  
                                                            
yeard2009                           0.0001                  
                                   (0.016)                  
                                                            
yeard2010                           -0.017                  
                                   (0.019)                  
                       

We can use tidy to get the results in a data frame for easy plotting

In [14]:
resreg3<-broom::tidy(reg3, conf.int = TRUE)

# Filter for those terms that start with a "d" and are followed by a digit
resreg3 <- resreg3  %>% filter(grepl("d\\d",term))
resreg3

term,estimate,std.error,statistic,p.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
d2007,1.456097e-02,0.01535425,0.948334561,0.3430376,-0.01554529,0.04466723
d2008,-5.803509e-03,0.01575791,-0.368291867,0.7126823,-0.03670125,0.02509423
d2009,6.498964e-05,0.01561102,0.004163063,0.9966787,-0.03054473,0.03067470
d2010,-1.725595e-02,0.01873382,-0.921112323,0.3570679,-0.05398878,0.01947688


#### Fixest

We can use the fixest package which is designed for estimating models with fixed-effects, particularly in an econometric context. It is highly efficient and is known for its speed in estimating large datasets. 

In [15]:
# Add a FE by Neighborhood and cluster the estandard errors by neighborhood
reg4<-feols(log_Sale_Price ~d2007+d2008+d2009+d2010+ Gr_Liv_Area  
            | Bldg_Type + factor(Neighborhood),vcov = ~Neighborhood,
            data=ames)

In [16]:
#results are shown with the etable function
etable(reg4)

,,reg4
,<chr>,<chr>
1,Dependent Var.:,log_Sale_Price
2,,
3,d2007,-0.0043 (0.0085)
4,d2008,0.0023 (0.0073)
5,d2009,-0.0167 (0.0128)
6,d2010,-0.0110 (0.0104)
7,Gr_Liv_Area,0.0004*** (2.83e-5)
8,Fixed-Effects:,-------------------
9,Bldg_Type,Yes
